In [1]:
import os
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import csv
from torchvision import models, transforms

In [2]:
class MyDigitDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        image_paths = []
        for ext in ('*.jpg', '*.jpeg', '*.png'):
            image_paths.extend(glob.glob(os.path.join(root_dir, '*', 'hand_written_digit', '*', ext)))
        self.image_paths = image_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = int(os.path.basename(img_path).split('_')[0])
        if self.transform:
            image = self.transform(image)
        return image, label
class MyTestDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.transform = transform
        self.image_paths = []
        for ext in ('*.jpg', '*.jpeg', '*.png'):
            self.image_paths.extend(glob.glob(os.path.join(folder, ext)))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        while idx < len(self.image_paths):
            img_path = self.image_paths[idx]
            try:
                image = Image.open(img_path).convert("RGB")
                if self.transform:
                    image = self.transform(image)
                return image, img_path
            except Exception:
                idx += 1  # Bỏ qua ảnh lỗi, thử ảnh tiếp theo
        raise IndexError("No valid image found at or after index {}".format(idx))

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),                   # Resize đúng input MobileNet
    transforms.ToTensor(),                           
   
     

])


In [4]:
root_path = r'C:\Users\Admin\Desktop\python\doan'
traindata = MyDigitDataset(root_path, transform=transform)
dataloader = DataLoader(traindata, batch_size=64, shuffle=True, num_workers=0)

test_folder = r'C:\Users\Admin\Desktop\python\MUOINGAN'
testdata = MyTestDataset(test_folder, transform=transform)
test_loader = DataLoader(testdata, batch_size=64,num_workers=0, shuffle=False)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mobilenet_v2(pretrained=False) # true = trong số đã train, false = tự train
model.classifier[1] = nn.Linear(model.last_channel, 10)  # 10 chữ số (0–9)
model.to(device)

c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [6]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):  # tăng nếu muốn
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/10] - Loss: {running_loss:.4f} - Acc: {100 * correct / total:.2f}%")


Epoch [1/10] - Loss: 224.7186 - Acc: 14.54%
Epoch [2/10] - Loss: 148.7560 - Acc: 48.58%
Epoch [3/10] - Loss: 73.9129 - Acc: 76.14%
Epoch [4/10] - Loss: 45.4344 - Acc: 85.98%
Epoch [5/10] - Loss: 33.4681 - Acc: 89.01%
Epoch [6/10] - Loss: 25.0935 - Acc: 92.16%
Epoch [7/10] - Loss: 22.5301 - Acc: 92.78%
Epoch [8/10] - Loss: 19.8431 - Acc: 93.65%
Epoch [9/10] - Loss: 15.7554 - Acc: 94.91%
Epoch [10/10] - Loss: 17.4047 - Acc: 94.19%


In [7]:
model.eval()
results = []

with torch.no_grad():
    for images, img_paths in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        preds = preds.cpu().numpy()  # chuyển về numpy để dễ xử lý

        for path, pred in zip(img_paths, preds):
            filename = os.path.basename(path)
            results.append([filename, int(pred)])

# Ghi ra file CSV
with open('MOBILENETV2_10K.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['filename', 'prediction'])  # header
    writer.writerows(results)